# cleaning oraganized dataframe

In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation

In [2]:
main_df = pd.read_csv('organized_dataframe.csv')

In [3]:
main_df

,date,content,likeCount,quoteCount,hashtags,twitter_id
0,2021-11-04 23:59:50+00:00,@Banterm42404691 @MrHarryCole Absolutely one s...,0,0,['COP26'],aquitainexox
1,2021-11-04 23:59:48+00:00,"Please join us at a #COP26 side event ""Impleme...",2,1,['COP26'],GEC_JCM_Info
2,2021-11-04 23:59:46+00:00,#India’s huge solar uptake has boosted #climat...,1,0,"['India', 'climate', 'COP26']",zpnine
3,2021-11-04 23:59:39+00:00,Reunión #COP26 Reunión mundial de países sobre...,0,0,"['COP26', 'AcuerdoDeParis', 'LenguajeClaro']",AsoPalabrasCl
4,2021-11-04 23:59:35+00:00,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...,0,0,"['COP26', 'PLNGreen']",Crstianst
...,...,...,...,...,...,...
7611,2021-11-04 13:20:36+00:00,"ClimateChange is no more only a challenge, it’...",5,0,"['Climate', 'ClimateAction', 'COP26', 'climate...",AqsaYounasRana
7612,2021-11-04 13:20:18+00:00,We confronted the coal baron this morning! Fuc...,3,0,"['JoeManchin', 'JoeManchinSenatorForSale', 'Jo...",pete4peace
7613,2021-11-04 13:19:58+00:00,Feeling simply disgusted by countries such as ...,0,0,"['china', 'usa', 'india', 'COP26Glasgow', 'Fos...",mmill_landscape
7614,2021-11-04 13:19:45+00:00,🚨 Join us in calling for @Google and #Facebook...,0,0,"['Facebook', 'climatechange', 'disinformation'...",KarynCaplan


In [4]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7616 entries, 0 to 7615
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        7616 non-null   object
 1   content     7616 non-null   object
 2   likeCount   7616 non-null   int64 
 3   quoteCount  7616 non-null   int64 
 4   hashtags    7616 non-null   object
 5   twitter_id  7616 non-null   object
dtypes: int64(2), object(4)
memory usage: 357.1+ KB


In [5]:
main_df.cp = main_df.copy()

<ipython-input-5-6d528d888e84>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  main_df.cp = main_df.copy()


In [6]:
id_tweet_df = main_df.cp[['twitter_id','content']]

In [7]:
id_tweet_df

,twitter_id,content
0,aquitainexox,@Banterm42404691 @MrHarryCole Absolutely one s...
1,GEC_JCM_Info,"Please join us at a #COP26 side event ""Impleme..."
2,zpnine,#India’s huge solar uptake has boosted #climat...
3,AsoPalabrasCl,Reunión #COP26 Reunión mundial de países sobre...
4,Crstianst,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...
...,...,...
7611,AqsaYounasRana,"ClimateChange is no more only a challenge, it’..."
7612,pete4peace,We confronted the coal baron this morning! Fuc...
7613,mmill_landscape,Feeling simply disgusted by countries such as ...
7614,KarynCaplan,🚨 Join us in calling for @Google and #Facebook...


## Appying cleaning fucntions to the tweets

In [11]:
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

     |████████████████████████████████| 24.0 MB 30.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 7.6 MB/s  eta 0:00:01


In [12]:
#import gensim
#from nltk.stem import WordNetLemmatizer
import re

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet)
    return tweet

def tokenize_tweets(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [13]:
#id_tweet_df
removed_links = []
for i in range(len(id_tweet_df['content'])):
    removed_links.append(remove_links(id_tweet_df['content'][i]))
id_tweet_df['removed_links'] = removed_links

<ipython-input-13-53f7133d7486>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_tweet_df['removed_links'] = removed_links


In [14]:
id_tweet_df['content'][0]

'@Banterm42404691 @MrHarryCole Absolutely one self indulgent ego trip!! #COP26'

In [15]:
id_tweet_df['removed_links'][0]

'@Banterm42404691 @MrHarryCole Absolutely one self indulgent ego trip!! #COP26'

In [16]:
removed_users = []
for i in range(len(id_tweet_df['content'])):
    removed_users.append(remove_users(id_tweet_df['removed_links'][i]))
id_tweet_df['removed_users'] = removed_users

<ipython-input-16-04bce3b4546d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_tweet_df['removed_users'] = removed_users


In [17]:
id_tweet_df['content'][0]

'@Banterm42404691 @MrHarryCole Absolutely one self indulgent ego trip!! #COP26'

In [18]:
id_tweet_df['removed_users'][0]

'  Absolutely one self indulgent ego trip!! #COP26'

In [19]:
removed_hashtags = []
for i in range(len(id_tweet_df['content'])):
    removed_hashtags.append(remove_hashtags(id_tweet_df['removed_users'][i]))
id_tweet_df['removed_hashtags'] = removed_hashtags

<ipython-input-19-3b49b71db1ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_tweet_df['removed_hashtags'] = removed_hashtags


In [20]:
id_tweet_df['removed_hashtags'][0]

'  Absolutely one self indulgent ego trip!! '

In [21]:
removed_av = []
for i in range(len(id_tweet_df['content'])):
    removed_av.append(remove_av(id_tweet_df['removed_hashtags'][i]))
id_tweet_df['removed_av'] = removed_av

In [22]:
id_tweet_df['removed_av'][0]

'  Absolutely one self indulgent ego trip!! '

In [23]:
tokens = []
for i in range(len(id_tweet_df['content'])):
    tokens.append(preprocess_tweet(id_tweet_df['content'][i]))
id_tweet_df['tokens'] = tokens

In [25]:
id_tweet_df['tokens'][0]

'absolutely self indulgent ego trip'

In [26]:
lemmatized = []
for i in range(len(id_tweet_df['tokens'])):
    lemmatized.append(lemmatize(id_tweet_df['tokens'][i]))
id_tweet_df['lemmatized'] = lemmatized

In [27]:
id_tweet_df['lemmatized'][0]

'absolutely self indulgent ego trip'

In [31]:
'''def preprocess(text):
    
    result = []
    
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    
    return result'''

In [42]:
#preprocess(list(id_tweet_df['removed_av']))

In [43]:
'''preprocess = []
for i in range(len(id_tweet_df['lemmatized'])):
    preprocess.append(preprocess(id_tweet_df['lemmatized'][i]))
id_tweet_df['preprocess'] = preprocess'''

"preprocess = []\nfor i in range(len(id_tweet_df['lemmatized'])):\n    preprocess.append(preprocess(id_tweet_df['lemmatized'][i]))\nid_tweet_df['preprocess'] = preprocess"

In [36]:
id_tweet_df.head()

,twitter_id,content,removed_links,removed_users,removed_hashtags,removed_av,tokens,lemmatized
0,aquitainexox,@Banterm42404691 @MrHarryCole Absolutely one s...,@Banterm42404691 @MrHarryCole Absolutely one s...,Absolutely one self indulgent ego trip!! #COP26,Absolutely one self indulgent ego trip!!,Absolutely one self indulgent ego trip!!,absolutely self indulgent ego trip,absolutely self indulgent ego trip
1,GEC_JCM_Info,"Please join us at a #COP26 side event ""Impleme...","Please join us at a #COP26 side event ""Impleme...","Please join us at a #COP26 side event ""Impleme...","Please join us at a side event ""Implementing ...","Please join us at a side event ""Implementing ...",join event implement joint credit mechanism jc...,join event implement joint credit mechanism jc...
2,zpnine,#India’s huge solar uptake has boosted #climat...,#India’s huge solar uptake has boosted #climat...,#India’s huge solar uptake has boosted #climat...,"’s huge solar uptake has boosted goals, says ...","’s huge solar uptake has boosted goals, says ...",huge solar uptake boost goals say minister,huge solar uptake boost goals say minister
3,AsoPalabrasCl,Reunión #COP26 Reunión mundial de países sobre...,Reunión #COP26 Reunión mundial de países sobre...,Reunión #COP26 Reunión mundial de países sobre...,Reunión Reunión mundial de países sobre cambi...,Reunión Reunión mundial de países sobre cambi...,reunión reunión mundial países sobre cambio cl...,reunión reunión mundial países sobre cambio cl...
4,Crstianst,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...,@_pln_id PLN terdepan dukung pemanfaatan EBT🍃\...,pln pln terdepan dukung pemanfaatan ebt,pln pln terdepan dukung pemanfaatan ebt


In [37]:
#lem_list = id_tweet_df['lemmatized'][0].split()

In [38]:
#lem_list

['absolutely', 'self', 'indulgent', 'ego', 'trip']

## Finding all the documents in english

In [45]:
!pip install langdetect
import string
from langdetect import detect
import pickle

     |████████████████████████████████| 981 kB 6.7 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=61823965434f640b418095642f1447f8f9a9b4bb46c2a15cedca36d7a28f1aa6
  Stored in directory: /Users/amirreza/Library/Caches/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect


In [46]:
def detectlang(s):
    try:
        return detect(s)
    except:
        return np.nan

In [48]:
id_tweet_df.loc[12]

twitter_id                                                  OwlHoot16
content             The CLIMATE CONFERENCE said #coal\nIs no longe...
removed_links       The CLIMATE CONFERENCE said #coal\nIs no longe...
removed_users       The CLIMATE CONFERENCE said #coal\nIs no longe...
removed_hashtags    The CLIMATE CONFERENCE said \nIs no longer KIN...
removed_av          The CLIMATE CONFERENCE said \nIs no longer KIN...
tokens              climate conference say longer king special lit...
lemmatized          climate conference say longer king special lit...
Name: 12, dtype: object

In [49]:
content = id_tweet_df.loc[12].content
print(detectlang(content))
print(content)

en
The CLIMATE CONFERENCE said #coal
Is no longer KING and our very special little PRIME MINISTER SCOTT MORRISON said fuck...    #ClimateAction #COP26
#Coal
#Glasgow 
AUKUS 
#France 
#Submarines
#Australia 
#auspol


In [50]:
id_tweet_df['content'] = id_tweet_df.content.astype(str)
id_tweet_df['lang'] = id_tweet_df['content'].map(detectlang)

In [52]:
id_tweet_df.lang.value_counts()

en       5954
es        656
fr        169
nl        125
it        119
de        114
cy         80
pt         61
ja         44
id         33
ca         32
pl         29
hi         27
tr         19
ar         18
fi         13
af         13
ro         12
tl         12
et          8
kn          7
sv          6
sw          6
el          5
ta          5
fa          5
ur          4
cs          4
ru          3
ko          3
hu          3
gu          3
no          3
da          3
th          3
he          2
hr          2
so          2
sl          2
ne          2
mr          1
te          1
vi          1
zh-cn       1
mk          1
Name: lang, dtype: int64

In [53]:
# pulling only English article to reduce noise in NLP
english_id_tweet_df = id_tweet_df[id_tweet_df.lang=='en']

In [55]:
english_id_tweet_df.shape

(5954, 9)

In [57]:
english_id_tweet_df.to_csv("english_clean_df.csv", index=False)

In [58]:
english_id_tweet_df

,twitter_id,content,removed_links,removed_users,removed_hashtags,removed_av,tokens,lemmatized,lang
0,aquitainexox,@Banterm42404691 @MrHarryCole Absolutely one s...,@Banterm42404691 @MrHarryCole Absolutely one s...,Absolutely one self indulgent ego trip!! #COP26,Absolutely one self indulgent ego trip!!,Absolutely one self indulgent ego trip!!,absolutely self indulgent ego trip,absolutely self indulgent ego trip,en
1,GEC_JCM_Info,"Please join us at a #COP26 side event ""Impleme...","Please join us at a #COP26 side event ""Impleme...","Please join us at a #COP26 side event ""Impleme...","Please join us at a side event ""Implementing ...","Please join us at a side event ""Implementing ...",join event implement joint credit mechanism jc...,join event implement joint credit mechanism jc...,en
2,zpnine,#India’s huge solar uptake has boosted #climat...,#India’s huge solar uptake has boosted #climat...,#India’s huge solar uptake has boosted #climat...,"’s huge solar uptake has boosted goals, says ...","’s huge solar uptake has boosted goals, says ...",huge solar uptake boost goals say minister,huge solar uptake boost goals say minister,en
5,PGDynes,The concrete needed to accommodate 2 billion m...,The concrete needed to accommodate 2 billion m...,The concrete needed to accommodate 2 billion m...,The concrete needed to accommodate 2 billion m...,The concrete needed to accommodate 2 billion m...,concrete need accommodate billion people citie...,concrete need accommodate billion people citie...,en
7,Xplorit,#WestVirginia Senator &amp; Ennersystams fraud...,#WestVirginia Senator &amp; Ennersystams fraud...,#WestVirginia Senator &amp; Ennersystams fraud...,Senator &amp; Ennersystams fraud . gets a pro...,Senator &amp; Ennersystams fraud . gets a pro...,senator amp ennersystams fraud get proper welc...,senator amp ennersystams fraud get proper welc...,en
...,...,...,...,...,...,...,...,...,...
7611,AqsaYounasRana,"ClimateChange is no more only a challenge, it’...","ClimateChange is no more only a challenge, it’...","ClimateChange is no more only a challenge, it’...","ClimateChange is no more only a challenge, it’...","ClimateChange is no more only a challenge, it’...",climatechange challenge crisis soon catastroph...,climatechange challenge crisis soon catastroph...,en
7612,pete4peace,We confronted the coal baron this morning! Fuc...,We confronted the coal baron this morning! Fuc...,We confronted the coal baron this morning! Fuc...,We confronted the coal baron this morning! Fuc...,We confronted the coal baron this morning! Fuc...,confront coal baron morning fuck,confront coal baron morning fuck,en
7613,mmill_landscape,Feeling simply disgusted by countries such as ...,Feeling simply disgusted by countries such as ...,Feeling simply disgusted by countries such as ...,Feeling simply disgusted by countries such as ...,Feeling simply disgusted by countries such as ...,feel simply disgust countries sign decrease st...,feel simply disgust countries sign decrease st...,en
7614,KarynCaplan,🚨 Join us in calling for @Google and #Facebook...,🚨 Join us in calling for @Google and #Facebook...,🚨 Join us in calling for and #Facebook () to ...,🚨 Join us in calling for and () to stop prom...,🚨 Join us in calling for and () to stop prom...,join call stop promote fund climate denial pub...,join call stop promote fund climate denial pub...,en
